### Starting your first 3Di simulation

In this notebook we will go through the steps of connecting to the threedi-api and
start our first simple 3Di simulation.

Let's first import all required modules for the notebook:

In [ ]:
from datetime import datetime
from getpass import getpass
from threedi_api_client.openapi import ApiException
from threedi_api_client.api import ThreediApi
from threedi_api_client.openapi.api.v3_api import V3Api

Then we'll provide our credentials to connect to the threedi-api

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = getpass("Personal API token")  # https://management.3di.live/personal_api_keys

config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)

Check if we can connect to the threedi-api with the provided credentials:

In [ ]:
try:
    user = api_client.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

To run a simulation we'll need a threedi-model. Let's see which threedi-models are
available:

In [ ]:
models = api_client.threedimodels_list(limit=5)  # limit to the first 5 results
for model in models.results:
    print(f"{model.name}")

If we already know the name of a specific model, we can also look it up with the following api call:

In [ ]:
my_model = api_client.threedimodels_list(name__icontains='v2_bergermeer')
print(my_model)

my_model = my_model.results[0]

Now that we have a model we are almost ready to create the simulation. But first we'll need 
to get an organisation under which's name we will run the simulation.

Let's see which organisations are available:

In [ ]:
organisations = api_client.organisations_list()
for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

# Here I use the following organisation, you should select from the list below
organisation_uuid = "b08433fa47c1401eb9cbd4156034c679"

Using a simulation template (with filled in settings and initials) makes it easier to start a new simulation. Let's find
a simulation template for the 3Di model.

In [ ]:
simulation_templates = api_client.simulation_templates_list(simulation__threedimodel__id=my_model.id)

for simulation_template in simulation_templates.results:
    print(simulation_template.id, simulation_template.name)

assert simulation_templates.count > 0, f"No simulation templates found for threedimodel {my_model.name}"

simulation_template_id = simulation_templates.results[0].id

Now we got all the pieces to create a simulation based on a simulation template

In [ ]:
my_simulation = api_client.simulations_from_template(
    data={
        "template": simulation_template_id,
        "name": "my first simulation",
        "organisation": organisation_uuid,
        "start_datetime": datetime.now(),
        "duration": 3600  # in seconds, so we simulate for 1 hour
    }
)

print(f"Created simulation with id: {my_simulation.id}")

The simulation has now been created but is not yet running. We can see the status of
the simulation with the following call:

In [ ]:
status = api_client.simulations_status_list(my_simulation.id)
print(status)

To run this simulation we have to tell it to start:

In [ ]:
api_client.simulations_actions_create(my_simulation.id, data={"name": "start"})

Eventually the simulation will finish. You can periodically check its progress by calling
for the status again:

In [ ]:
api_client.simulations_status_list(my_simulation.id)


Eventually you should see something like this::

    {
        'created': datetime.datetime(2020, 7, 27, 14, 7, 6, 654905, tzinfo=tzutc()),
        'id': 15866,
        'name': 'finished',
        'paused': False,
        'time': 3600.0
     }

Congratulations, you just made your first simulation via the threedi-api-client!